In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
import os

from taogpt.orchestrator import *
from taogpt.utils import *
from taogpt.llm_model import LangChainLLM
from taogpt.prompts import PromptDb

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
TEMPERATURE = 0.7

In [5]:
with open(os.path.join(os.environ['HOME'], '.ssh', 'openai-zillow.json'), 'r') as f:
    credentials = json.load(f)
os.environ["OPENAI_API_KEY"] = credentials['key']
os.environ["OPENAI_API_BASE"] = credentials['url']
llm3_5 = ChatOpenAI(model_name='gpt-3.5-turbo-16k', temperature=TEMPERATURE)
llm4 = ChatOpenAI(model_name='gpt-4', temperature=TEMPERATURE)
llm4_32k = ChatOpenAI(model_name='gpt-4-32k', temperature=TEMPERATURE)

conversation = ConversationChain(llm=llm3_5)
conversation.predict(input="What's your model version?")

"I am using OpenAI's GPT-3 model, which is the third iteration of the Generative Pre-trained Transformer model. It was released in June 2020 and has been trained on a vast amount of text data to generate human-like responses. GPT-3 has 175 billion parameters, making it one of the largest language models ever created."

In [6]:
llm3_5.model_name, llm4.model_name, llm4_32k.model_name

('gpt-3.5-turbo-16k', 'gpt-4', 'gpt-4-32k')

In [21]:
prompts = PromptDb.load_defaults()
logger = MarkdownLogger('logs/taogpt_log.md')
config = Config(
    ask_user_before_execute_codes=False,
    ask_user_questions_in_one_prompt=True,
    pause_after_initial_solving_expansion=True,
    first_expansion=1,
    initial_expansion=2,
    max_tree_branches=6,
    max_tokens=5000,
    check_final=True
)
executor = Orchestrator(
    config=config,
    llm=LangChainLLM(llm3_5, logger=logger),
    prompts=prompts,
    markdown_logger=logger,
    sage_llm=LangChainLLM(llm4, logger=logger),
)

experiment_name = 'example'
executor.start("""
What's x mod (y - z) where x = 10, y = 7, z = 4.
""")
# executor.start("""
# Solve this 4x4 Sudoku.
#
# ```text
# _ 3 _ 1
# 1 _ _ 3
# 2 _ 3 _
# 3 4 _ 2
# ```
#
# (`_` represents an empty cell.)
#
# Solve in your head. Do NOT use Python programming.
# """, analyze_first=True)

Pause: Pause after initial solving expansion

In [12]:
# executor._chain[11].step.description

In [13]:
# executor._chain = executor._chain[:11]

In [14]:
# executor.chain[-1].step.description

In [15]:
# executor._prompts = PromptDb.load_defaults()

In [22]:
executor.resume(5000, additional_tokens_for_smarter_llm=0, unblock_initial_expansion=True)

In [19]:
logger = MarkdownLogger(f'examples/{experiment_name}.final.md')
executor.log_configs(logger)
logger.log_conversation(executor.show_conversation_thread(with_header=True))
logger.log(f"**total tokens**: {executor.llm.total_tokens}")
